#  Recommender Systems


**Author: Carleton Smith**

**Reviewer: Jessica Cervi**

**Expected time = 2.5 hours**

**Total points = 90 points**
 
## Assignment Overview

In this assignment, we will work on recommender systems. Recommender systems provide a non-parametric comparison between items. They are fundamental in analyzing how individuals can be served.
This project consists of 4 parts:  
 
 - General Familiarization with the data.
 - Mathematical foundations of recommender systems and simple examples about them.
 - Execution of methods on the data.
 - Short introduction to the `Surprise` package.   
 
All the methods used below should be familiar from the material from this week's lectures. Except for the singular value decomposition (SVD) and the `Surprise` package, we will review some main concepts before demonstration.

The general goal of this project is, given a collection of users, items, and user reviews of the items, predict what score a user would assign to an item they have yet to review.  

We will be working with a synthetic review dataset, modeled on reviews from Amazon. To demonstrate the techniques, in all our examples, we will work only with a smaller portion of the made-up data.

This assignment is designed to build your familiarity and comfort in coding in Python. It will also help you review the key topics from each module. As you progress through the assignment, answers to the questions will get increasingly complex. You must adopt a data scientist's mindset when completing this assignment. Remember to run your code from each cell before submitting your assignment. Running your code beforehand will notify you of errors and giving you a chance to fix your errors before submitting it. You should view your Vocareum submission as if you are delivering a final project to your manager or client. 

***Vocareum Tips***
- Do not add arguments or options to functions unless asked specifically. This will cause an error in Vocareum.
- Do not use a library unless you are explicitly asked in the question. 
- You can download the Grading Report after submitting the assignment. It will include the feedback and hints on incorrect questions. 




### Learning Objectives
- Understand mathematical foundations of recommender systems
- Translate a mathematical algorithm into code
- Determine similarity between items by using Euclidean distance, Pearson's coorelation-coefficient, and cosine similiarty 
- Predict item recommendations using similarity scores 
- Understand utilization complexity to noramlize user ratings 
- Make item recommendations using singular value decomposition (SVD) 
- Implement SVD using the `surprise` package


## Index: 

###  Recommender Systems

+ [Question 01](#q1)
+ [Question 02](#q2)
+ [Question 03](#q3)
+ [Question 04](#q4)
+ [Question 05](#q5)
+ [Question 06](#q6)
+ [Question 07](#q7)
+ [Question 08](#q8)
+ [Question 09](#q9)
+ [Question 10](#q10)
+ [Question 11](#q11)




#  Recommender Systems

### Importing the dataset 
In the first part of this assignment, we will be using synthetic data modeled from Amazon reviews. A description of the review data and instructions to download the database [can be found here](https://s3.amazonaws.com/amazon-reviews-pds/readme.html).  

We begin by importing the libraries we will use in this assignment.

Next, we use the `pandas` `read_table` function to read the dataset. Finally, we display a sample of the data taken from Amazon.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

%matplotlib inline

In [ ]:
sample_data_path = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/sample_us.tsv"

rev_df = pd.read_csv(sample_data_path, sep='\t')

rev_df.head()

### Looking at our raw data

Although Natural Language Processing (NLP) might offer insight into the structure of reviews, for this assignment, we are only interested in the customers (`customer_id`), the products (`product_id`), and the scores (`star_rating`) that each customer assigned to a particular product.


[Back to top](#Index:) 
<a id='q1'></a>

### Question 1:

*5 points*

Extract a subset from our dataframe `rev_df` containing only the columns `customer_id`,`product_id` and `star_rating`. Assign the new dataframe to `rev_df`. Finally, rename the column `star_rating` to `score`.

In [ ]:
### GRADED

### YOUR SOLUTION HERE

###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


Let's have a look at our new dataframe!

In [ ]:
rev_df.head()

Although this is not strictly necessary, we can modify the values in columns customer_id and product_id to make them a little easier to read.  

We will do so by building two dictionaries: one for `customer_id` and the other one for `product_id` in the following way.

Let the keys for your dictionary be all the *n unique* values of `customer_id`. Rewrite such values to read "R" followed by a number ###, from 0 to n, such that each unique key is mapped to a unique string of the format `R###`. We do the same for the products, but using "P" as a prefix instead.

In [ ]:
rID_dict = {rID:'R'+ str(idx) for idx, rID in enumerate(rev_df['customer_id'].unique()) }
pID_dict = {pID:'P'+ str(idx) for idx, pID in enumerate(rev_df['product_id'].unique()) }


### Checking the values in the dictionaries below for products.
for k in list(pID_dict.keys())[:5]:
    print(k, ":",pID_dict[k])
print("length: ",len(pID_dict))

[Back to top](#Index:) 
<a id='q2'></a>

### Question 2:

*10 points*


Replace the values in the column `product_id` in the dataframe `ref_df` with the entries in the dictionary `pID_dict` created above. Do the same with the `customer_id` column, but, this time, use the entries in `pID_dict`.

**HINT:** You can achieve this by using the `loc` and `map` functions.

In [ ]:
### GRADED

### YOUR SOLUTION HERE

###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


Let's have a look at our new dataframe!

In [ ]:
rev_df.head()

<a id = "Exploratory Data Analysis"></a>
### Exploratory Data Analysis (EDA)
Having gone through the process of cleaning and analyzing the data, we will load a more extensive synthetic dataset and perform some light EDA to look at the distributions of the data.

In [ ]:
# Reading the data
synth_data_path = "./data/synthetic_reviews.csv"

rev_df = pd.read_csv(synth_data_path)
rev_df.head()

First, we visualize how many products got a score of 5, 4, 3, 2, or 1. 

In [ ]:
rev_df['score'].value_counts().plot(kind = 'bar');

Next, we count how many products received a specific score.

In [ ]:
rev_df['score'].value_counts()

We notice that there is an interesting preponderance of products with score 5. 

We continue by ranking the product from the ones that had the most reviews to the ones that had the least. We display the first 15 and the last 5 rows of this ranking.

In [ ]:
print("Reviews per product, top and bottom reviewed\n")
#Display first 15 rows
print(rev_df['productID'].value_counts()[:15])
#Display last 5 rows
print(rev_df['productID'].value_counts()[-5:])

Below, we visualize the ranking defined above.

In [ ]:
product_w_n_reviews = rev_df['productID'].value_counts().value_counts()
plt.figure(figsize = (10,7))
plt.scatter(product_w_n_reviews.index, product_w_n_reviews)
plt.xlabel("Number of Reviews", fontsize = 16)
plt.ylabel("Number of Products", fontsize = 16);

The above graph seems to be on a log-normal scale. 

In reality, the graph displayed has the shape of a "hollow curve". The log-normal appearance is an artifact of the data synthesis.

We will continue our EDA by ranking which customers gave the most and least number of reviews.

In [ ]:
print("Reviews per reviewer, top and bottom reviewers\n")
print(rev_df['reviewerID'].value_counts()[:15])
print(rev_df['reviewerID'].value_counts()[-5:])

Again, we visualize the ranking defined above.

In [ ]:
product_w_n_reviews = rev_df['reviewerID'].value_counts().value_counts()
plt.figure(figsize = (10,7))
plt.scatter(product_w_n_reviews.index, product_w_n_reviews)
plt.xlabel("Number of Reviews", fontsize = 16)
plt.ylabel("Number of Users", fontsize = 16);

As expected, most products are only reviewed a few times, most users only submit a few reviews, and very few users have submitted many reviews. 

Again. the above graph has the shape of a "hollow curve".

<a id = "simple"></a>
### Functioning of Recommender Systems

Before tackling our extensive synthetic data, we will use a simpler dataset to demonstrate the functioning of recommender systems.  

Below we have a set of rankings of six musicians made by six individuals.  

In [ ]:
example_path = "./data/example.csv"
ex = pd.read_csv(example_path, index_col = 0)
ex

Note that the value "0" in the rows above denotes the absence of ranking - **NOT** an extremely poor ranking.  

To determine which product to recommend, we can compute the similarity scores between products. Similarity scores can be computed in three different ways:

- the Euclidean distance,
- the Pearson's correlation coefficient,
- and the cosine similarity score. 

To compare these scores we will normalize each to range between 0 and 1.  
To simplify our analysis, we will use a subset of our example DataFrame:

In [ ]:
ex_hand = ex.iloc[:3,-3:]
ex_hand

### Euclidean Distance

The Euclidean distance between point $i =(a_i, b_i, \dots, n_i)$ and point $j= (a_j, b_j, \dots, n_j)$ is given by

$$\sqrt{(a_j-a_i)^2+(b_j-b_i)^2+...+(n_j-n_i)^2}$$  


[Back to top](#Index:) 
<a id='q3'></a>

### Question 3:

*5 points*

Calculate the Euclidean distance between Brahms and Wagner in the `ex_hand` DataFrame. 


Assign the computed distance as a float to the variable ans2. Make sure that the answer is accurate to 3 decimal places.


In [ ]:
### GRADED

### YOUR SOLUTION HERE

ans2 = None
###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


To normalize the Euclidean distance between 0 and 1, we can use the following formula that relates "distance" to "similarity"

$$\text{similarity} =\frac1{1+\text{distance}}.$$

For example, a distance of 0 corresponds to a similarity of 1, and a very large distance (e.g., $\infty$) corresponds to a similarity of 0.  

[Back to top](#Index:) 
<a id='q4'></a>

### Question 4:

*10 points*

Define a  function called `e_sim` that accepts two `pandas` series as arguments.

Your function should return the Euclidean similarity score between the two series. Make sure that the answer is accurate to 3 decimal places.

For this question, you will need to use the attribute `np.linalg.norm()`.

In [ ]:
### GRADED


### YOUR SOLUTION HERE

def e_sim(ser1, ser2):
    """
    Given two Pandas series, compute the euclidean similarity score;
        1 / 1+euclidean distance
        
    Positional Arguments --
        ser1: a Pandas Series of length n
        ser2: a Pandas Series of length n
    
    Example --
        ser1 = ex_hand.iloc[:,0]
        ser2 = ex_hand.iloc[:,1]
        print(e_sim(ser1, ser2)) #--> 0.28989794855663564
    """
    return
###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


### Pearson's Correlation Coefficient

The Pearson's correlation coefficient $\rho$ takes values from -1 to 1. Therefore, to normalize it into a difference score between 0 and 1, we need to reduce that range by half (i.e., divide the range by two) to increase the minimum score to 0, and add .5:  

$$\frac12 + \frac{\rho_{xy}}2$$  

[Back to top](#Index:) 
<a id="q5"></a>
### Question 05

*10 points*

Define a  function called `p_sim` that accepts two `pandas` series as arguments.

Your function should return the Pearson correlation coefficient between the two series. Make sure that the answer is accurate to 3 decimal places.

For this question you will need to use the attribute `np.corrcoef()`.



In [ ]:
### GRADED


### YOUR SOLUTION HERE

def p_sim(ser1, ser2):
    """
    Given two Pandas series, compute the Pearson correlation coefficient;
    """
    return


###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


### Cosine Similarity

The cosine similarity calculates the cosine of the angle between two vectors normalizing it between 0 and 1. 

The cosine similarity score is given by the following formula

$$cos(\theta) = \frac{S1\cdot S2}{||S1||\cdot||S2||},$$
 
 where $S1$ and $S2$ are the two vectors considered, and $||\cdot||$ represents the magnitude of each vector.

[Back to top](#Index:) 
<a id="q6"></a>
### Question 06

*10 points*

Define a  function called `c_sim` that accepts two `pandas` series as arguments.

Your function should return the cosine similarity between the two series. Make sure that the answer is accurate to 3 decimal places.

For this question, you will need to use the attribute `np.linalg.norm()`.

In [ ]:
### GRADED

### YOUR SOLUTION HERE

def c_sim(ser1, ser2):
    """
    Function to compute the cosine similarity
    """
    
    return 
###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


At this point, we have defined three functions for calculating similarity between two vectors.  

- e_sim: for the Euclidean similarity 
- p_sim: for the Pearson correlation coefficient similarity
- c_sim: for the cosine similarity
 

Now, let's calculate each of the three similarity scores for each pair of musicians in the `ex` DataFrame.  

Let's recall what the DataFrame `ex` looks like.

In [ ]:
ex

The function we will build must take into account that not all users have rated all musicians.

So, for example, when comparing "Mozart" and "Bach", we must disregard the observations from "Abel" and "Baker", respectively.  

This can be done by reading the value "0" as to be "less than 1", and *not* as the lack of information which it actually represents.  

[Back to top](#Index:) 
<a id="q7"></a>
### Question 07

*10 points*

Define a  function called `drop_val` that accepts the following  arguments:

- A `pandas` DataFrame.
- A numeric variable named `to_drop`.

Your function should drop all the rows that have the value contained in `to_drop` in any column, and should return the DataFrame without the rows that have been dropped.

For example, if we pass the DataFrame `ex` containing all rows but only the columns corresponding to "Mozart" and "Bach", and we set  'to_drop=5', our function should return "Abel", "Erik", and "Frank" rows with all the original columns. 

In [ ]:
### GRADED

### YOUR SOLUTION HERE

def drop_val (df, to_drop):
    
    """
    Drop rows from the DataFrame containing the specified values
    
    Positional Arguments --
        df: a Pandas DataFrame
        to_drop: a value found in some rows of df
    
    Example --
        
        df = ex.loc[:,"Mozart":"Bach]
        to_drop = 5
        print(drop_val(df,to_drop)) # -->          Mozart  Bach
                                            Abel        0     1
                                            Erik        3     3
                                            Frank       2     2   
    """
    return
###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


The cell below defines a function "drop_rows_with_zeros" that takes a DataFrame with two columns and returns a DataFrame where all the rows that contain a value of 0 have been removed.

In [ ]:
### Defining drop_rows_with_zeros

def drop_rows_with_zeros(df):
    keep = np.intersect1d( df.iloc[:,0].to_numpy().nonzero(), df.iloc[:,1].to_numpy().nonzero())
    
    return df.iloc[keep,:]


### Comparison of Similarity Scores

Below, we will compare the similarity scores for each combination of musicians. 

First, we find all the possible pairs of musicians:

In [ ]:
import itertools
for c in itertools.combinations(ex.columns,2):
    print(c)

Each tuple can be used as a subset in the DataFrame. For example, the subset for Mozart and Bach is given by:

In [ ]:
ex[list(('Mozart', 'Bach'))]

Next, we define a first dictionary that has as keys the pairs of musicians being compared.

In a second dictionary, we define the similarity scores, "Euclid", "Pearson", and "Cosine",  corresponding to each pair of musicians.

In [ ]:
sim_scores = dict()

for c in itertools.combinations(ex.columns, 2):
    df = drop_rows_with_zeros(ex[list(c)])
    ser1 = df.iloc[:,0]
    ser2 = df.iloc[:,1]
    scores = {"Euclid":e_sim(ser1, ser2), "Pearson":p_sim(ser1,ser2), "Cosine":c_sim(ser1,ser2)}
    key =", ".join(c)
    sim_scores[key] = scores
    
sims = pd.DataFrame.from_dict(sim_scores,orient = "index")
sims

The similarity scores can be visualized as follow

In [ ]:
### Note: x and y scales are not consistent in this visualization
sns.pairplot(sims );

In [ ]:
### Visualization with consistent x and y scales using `matplotlib`
plt.figure(figsize = (12,6))
for i, cols in enumerate([('Euclid','Pearson'), ("Euclid","Cosine"),("Pearson","Cosine")]):
    plt.subplot(1,3,i+1)
    
    plt.scatter(sims[cols[0]], sims[cols[1]])
    
    plt.xlabel(cols[0], fontsize = 14); plt.ylabel(cols[1], fontsize = 14)
    plt.xlim(0,1); plt.ylim(0,1)
    
plt.tight_layout()

<a id = "sim-note"></a>
### Differences in Similarity Scores


As we can see from the graphs above, it is possible to visualize a range of similarity scores given different techniques. These similarity scores will majorly impact how a recommender system performs. 

Refer to the lectures for the particular impacts and strengths of the different similarity calculations.

<a id = 'pred'></a>
### Predicting Unknown Scores

Assume that we don't know the score that Abel gave to Mozart. To do so, we must take into account that Mozart has a similarity score in relation to all the other musicians.

Similarly, Abel has given a score to all the other musicians.  

Therefore, Abel's predicted score for Mozart will be given by: the **sum** of Abel's score for every other musician **times** that musician's similarity score with respect to Mozart, **divided by** the sum of similarity scores.  

For example, we can predict Abel's score for Mozart by computing

$$\frac{\sum_{ mus = Bach}^{Liszt}A_{mus}*\text{simScore}(\text{Mozart, mus})}{\sum_{mus = Bach}^{Liszt}\text{simScore}{\text{(Mozart, mus)}}}$$ 

The commands below compute the similarity scores for Mozart.

In [ ]:
drop_rows_with_zeros(ex.iloc[:, :6])

In [ ]:
mozSimScores = {}
for mus in ex.columns[1:]:
    no_zeros = drop_rows_with_zeros(ex[['Mozart', mus]])
    
    mozSimScores[mus] = round(p_sim(no_zeros.iloc[:,0], no_zeros.iloc[:,1]),2)

print(mozSimScores)

[Back to top](#Index:) 
<a id="q8"></a>
### Question 08

*10 points*

Given the similarity scores calculated and stored in `mozSimScores`, use the above formula to predict Abel's score for Mozart. Assign your result to the variable `ans6`. Make sure the answer is accurate to 3 decimal places.

*Hint*: Use the rounded score computed above.

In [ ]:
### GRADED


###  YOUR SOLUTION HERE

ans6 = None
###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


We observe that, although many other users gave Mozart high scores, because Abel appears to be less generous in his reviews, his predicted score for Mozart is fairly low. 

The function defined below predicts an unknown score based on the formula above.

In [ ]:
### Creating a function to create predictions in the manner described above.
### Note: "user" is a row, and "item" is a column in the df.
### simFunc defaults to p_sim

def scorePred(df, user, item, simFunc = p_sim, rev_items = None):
    """
    Positional Arguments --
        df: Pandas DataFrame
        user: Row-index in df
        item: Column-index in df
        simFunc: a similarity function (e_sim, p_sim, or c_sim)
        rev_items: For larger dfs, specifies all items reviewed by "user"
    """
    # Check to see if user has already scored item
    if df.loc[user,item] > 0:
        return "Already rated a "+str(df.loc[user,item])
    
    # rev_items used for larger DataFrame,
    # when you have all the items a particular user has reviewed already
    # otherwise, if "None" (if statement below)
    # take all other items other than item to predict
    if not rev_items:
        rev_items = set(df.columns)
        rev_items.remove(item)
    
    sim_total, user_sim_total = 0,0
    
    for other_item in rev_items:
        user_score_other_item = df.loc[user, other_item] # grab user score
        
        if user_score_other_item == 0:
            print("no user score")
            continue
        
        # Use function built above to drop all other users with "other_item" score of "0"
        no_zeros = drop_rows_with_zeros(df[[item, other_item]]) 
        sh = no_zeros.shape
        
        # If no other users, move to next item
        if sh[0] == 0:
            continue
            
        #print(no_zeros.shape)
        
        # calculate similarity score using non-zero information. 
        ser1 = no_zeros.iloc[:,0]
        ser2 = no_zeros.iloc[:,1]
        # print(ser1, ser2)
        ss = simFunc(no_zeros.iloc[:,0], no_zeros.iloc[:,1])
        # print(ss, user_score_other_item)
        
        # add up sim total and weighted sim total
        sim_total += ss
        user_sim_total += user_score_other_item * ss

        
    if sim_total == 0:
        return 0
    
    else:
        return user_sim_total / sim_total

In [ ]:
ex

Use the cells below to compute the unknown score for different cases using the function `scorePred` defined above.

In [ ]:
scorePred(ex, 'Baker', 'Bach')

<a id = "ret"></a>
### Returning to the Data
Let's go back to the Amazon rating DataFrame.

In [ ]:
rev_df.head(10)

We observe that now the data does not conform to the two-dimensional n-by-p matrix that we have been using. Therefore, we need to manipulate our data. 

[Back to top](#Index:) 
<a id="q9"></a>
### Question 09

*10 points*

Use the  `Pandas` built-in `pivot()` attribute to reshape our data. Set `reviewerID` as index and use `productID` to rename the columns. Assign the new dataframe to `reshaped_reviews`. Next, replace all the NaN values with zeroes. Finally, use `droplevel()` on the columns to remove the artifact of the pivot function.

In [ ]:
### GRADED


###  YOUR SOLUTION HERE


###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


Let's have a look at our reshaped dataframe.

In [ ]:
print("shape: ", reshaped_reviews.shape)
reshaped_reviews.head()

It should be clear from the output above that the majority of the entries corresponding to a certain pair of customer and product are now empty. Thus, when trying to provide recommendations, it will not be possible to predict scores for every  combination of customer and product. In fact, to predict scores, we require a certain amount of overlap between the data.

Below predictions are created for our reviewer called "R0"

In [ ]:
### First, find the products that 'R0' has reviewed.
### These are the products that will be used for creating similarity scores.

r0 = reshaped_reviews.loc["R0", :]
# numpy function returns all indexes that are not "0"
r0_nonzero = r0.to_numpy().nonzero()
# collect names of reviewed products
r0_rev_prods = list(r0.iloc[r0_nonzero].index) 
print(len(r0_rev_prods))
r0_rev_prods[:5]

Next, we create a list containing the product that have not been reviewed by R0.

In [ ]:
### Creating a list of products not reviewed by R0

# return list of all prods not in the "reviewed" list
r0_not_rev = np.setdiff1d(reshaped_reviews.columns, r0_rev_prods) 

print(len(r0_not_rev))
r0_not_rev

The commands below require too much processing for Vocareum. 
```python
### predict scores for products not yet reviewed by R0, using function built above: `scorePred`
### cosine similarity used because if a product all has only one value of review (e.g. all 5s)
### then the standard_deviation is 0 and the correlation coefficient is not calculable

preds = {}
for item in r0_not_rev:
   preds[item] = scorePred(reshaped_reviews, 'R0', item, simFunc = c_sim, rev_items = r0_rev_prods)

pd.Series(preds).sort_values(ascending = False).head(10)
```
We simply display the output below:

```
P470     5.0
P2342    5.0
P4839    5.0
P4796    5.0
P2703    5.0
P4800    5.0
P615     5.0
P4806    5.0
P2699    5.0
P1533    5.0
dtype: float64
```

<a id = "util"></a>
### Utilization Complexities

Certainly, if  a product is likely to score a "5", then that product should be recommended. However, because we are dealing with very sparse data, the confidence in our predictions will vary between each product.

[Back to top](#Index:) 
<a id="q10"></a>
### Question 10

*5 points*

Consider 3 products: `A`, `B`, and `X`. Assume that `userN` has ranked both products `A` and `B` with a score of 5. Assume that `user1` has ranked products `A` and `X` both with a score 3 and assume that `user2` has ranked products `B` and `X` with a score 2.

What will be the similarity score between product `A` and product `X`? Assign the result rounded to 2 decimal places the variable ans7a.

What will be the similarity score between product `B` and product `X`? Assign the result rounded to 2 decimal places the variable ans7b.


In [ ]:
### GRADED

### YOUR SOLUTION HERE

ans7a = None
ans7b = None
###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


It is important to note that we could have run into other possible scenarios: high similarity scores because of few overlapping reviews, or a high-user score for the user who is being predicted for.  

<a id = "svd"></a>
### Singular Value Decomposition (SVD)

We can deal with the sparsity of data and, at the same time, increase the speed of our algorithm,  by using Singular Value Decomposition (SVD). SVD is a technique that reduces the dimensionality of functions by "capturing" a majority of the information present in a dataset in a smaller number of variables.  

SVD relies upon some straight-forward linear algebra that is worth understanding. The main concepts have been covered in the lectures. Here are some more insightful resources about SVD.  

[Here is a nice introduction.](https://machinelearningmastery.com/singular-value-decomposition-for-machine-learning/)  

[Below, we will use `numpy's` implementation of SVD](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.linalg.svd.html)

We have learned that the quantity $\Sigma ^2$ describes the variance of each vector. The plot below displays the rate of the variance given by n vectors as a function of the total number of products.

![svd](./assets/svd.PNG)

We notice that about 1700 of the decomposed vectors will give around 80% of the variance in the data. Computationally, this will make the calculations much faster: in fact, considering only 1/3 of the vectors will yield 80% of the value. 

The commands below take too long to run for Vocareum. 
```python
# Perform SVD
u, sigma, _ = np.linalg.svd(reshaped_reviews)

# Create function to decompose DataFrame
def return_svd(df , u, sigma, n):
   sigN = np.mat(np.eye(n) * sigma[:n]) #arrange Sig4 into a diagonal matrix
   n_svd_vectors = np.dot(df.T, np.dot(u[:,:n] , sigN.I))  #create transformed items
   return pd.DataFrame(n_svd_vectors).T

# use function to create decomposed DataFrame
svd_df = return_svd(reshaped_reviews, u, sigma, 1700)
svd_df.columns = reshaped_reviews.columns

# Save DataFrame

svd_df.to_csv("svd_df.csv")
```
The results are loaded below:

In [ ]:
svd_df = pd.read_csv("./data/svd_df.csv", index_col = 0)
svd_df.head()

As `svd_df` indicates, the svd vectors correspond to different items. The calculation for the recommendation will be the same, but will be done by using these new values.  

For example, to predict the score that reviewer "R0" would give to  the product 'P1' (Remember "R0" already reviewed "P0") we must follow the following steps:  

1. Find the products that R0 scored.
2. Find the similarity between those scored products and "P1".
3. Multiply those similarity scores by the scores from R0.
4. Divide by sum of similarity scores.  

The downside of SVD is that it must be performed before the calculations, and, as was seen above, such computations can take some time. In fact, in practice, SVD would need to be performed each time a new review is added.

The following function computes the prediction scores from the `svd_df` DataFrame.

In [ ]:
### Creating a function to create predictions using the SVD_df
### Note: "user" is a row, and "item" is a column in the df, and a column in svd_df.
### simFunc defaults to p_sim

def scorePredSVD(df, user, item, svd_df, simFunc = p_sim, rev_items = None):
    
    # Check to see if user has already scored item
    if df.loc[user,item] > 0:
        return "Already rated a "+str(df.loc[user,item])
    
    # Code below should be familiar from "scorePred" defined above
    if not rev_items:
        rev_items = set(df.columns)
        rev_items.remove(item)
    
    sim_total, user_sim_total = 0,0
    
    for other_item in rev_items:
        user_score_other_item = df.loc[user, other_item]
        
        if user_score_other_item == 0:
            print("no user score")
            continue
            
        ser1 = svd_df.loc[:,item]
        ser2 = svd_df.loc[:,other_item]
        # print(ser1, ser2)
        ss = simFunc(ser1, ser2)
        # print(ss, user_score_other_item)
        sim_total += ss
        user_sim_total += user_score_other_item * ss

        
    if sim_total == 0:
        return 0
    
    else:
        return user_sim_total / sim_total

The commands below predict  the scores for products not yet reviewed by R0, using the function built above: scorePred.
We use the cosine similarity because if a product has only one value of review (e.g. all 5s), then the standard deviation is 0 and the correlation coefficient is not calculable.

In [ ]:
%%time

SVDpreds = {}
for item in r0_not_rev:
    SVDpreds[item] = scorePredSVD(reshaped_reviews, 'R0', item, svd_df, simFunc = c_sim, rev_items = r0_rev_prods)

In [ ]:
pd.Series(SVDpreds).sort_values(ascending = False).head()

<a id="q11"></a>
[Back to top](#Index:) 
### Question 11

*5 points*

SVD is a kind of:
- a) Variance reduction
- b) Special type of array in Python
- c) Dimensionality reduction
- d) Description of central tendency

Assign the character associated with your choice as a string to the variable ans8.

In [ ]:
### GRADED
### YOUR SOLUTION HERE

ans8 = None
###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


<a id = "surprise"></a>
### Surprise - Python Package
[`Surprise`](http://surpriselib.com) is a Python package which implements a number of tools for building and testing recommender systems. The documentation can be found [here](https://surprise.readthedocs.io/en/stable/).

**THE FOLLOWING CODE WILL NOT RUN ON VOCAREUM. WE JUST PROVIDE EXAMPLES IN CASE YOU CHOOSE TO USE `SURPRISE` IN YOUR OWN ENVIRONMENT**  


To install surprise, run:  

`conda install -c conda-forge scikit-surprise`  

in the Anaconda Prompt. After the installation, you might have to reinstall or update scipy, and restart Kernel by doing

`conda install -c anaconda scipy`  

Below is [example code from the surprise documentation](https://surprise.readthedocs.io/en/stable/getting_started.html#)

In [ ]:
# from surprise import SVD
# from surprise import Dataset
# from surprise.model_selection import cross_validate


# # Load the movielens-100k dataset (download it if needed),
# data = Dataset.load_builtin('ml-100k')

# # We'll use the famous SVD algorithm.
# algo = SVD()

# # Run 5-fold cross-validation and print results
# cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

[Loading in a df](https://surprise.readthedocs.io/en/stable/dataset.html?highlight=pandas#surprise.dataset.Dataset.load_from_df)

In [ ]:
# import os
# from surprise import BaselineOnly
# from surprise import Reader

# # when using a df to load in data, the columns must be:
# # user, product, rating, in that order
# # a reader must be defined to describe the rating_scale
# reader = Reader(rating_scale=(0,5))

# data = Dataset.load_from_df(rev_df, reader=reader)

# # We can now use this dataset as we please, e.g. calling cross_validate
# cross_validate(BaselineOnly(), data, verbose=True)

### One final note: 

This project has based its recommendations of comparing the similarity of different *items*.  Using the same technique, it is also possible to calculate the similarity of *users*. The reason why *items* is usually the default choice is that there are usually fewer items than users. This decreases the number of similarity calculations required. 

For example, in our data, we had roughly 3.5k items and 5.5k users. When using a complete dataset from a specific source, this difference will be more significant.